In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5749,2020-11-23T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1604,243,1847,36066,...,751,1249,15238,1227,33372.0,21006.0,54378,890048,611550.0,NaN
5750,2020-11-23T17:00:00,ITA,9,Toscana,43.769231,11.255889,1832,296,2128,50291,...,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0,effettuati 66132 tamponi rapidi antigenici di...
5751,2020-11-23T17:00:00,ITA,10,Umbria,43.106758,12.388247,371,78,449,10034,...,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0,NaN
5752,2020-11-23T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,146,13,159,1690,...,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5749,2020-11-23T17:00:00,19,Sicilia,1604,243,1847,36066,37913,751,1249,15238,1227,33372.0,21006.0,54378,890048,611550.0
5750,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0
5751,2020-11-23T17:00:00,10,Umbria,371,78,449,10034,10483,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0
5752,2020-11-23T17:00:00,2,Valle d'Aosta,146,13,159,1690,1849,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-23', '2020-11-22')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-23T17:00:00,1,Piemonte,5112,399,5511,71412,76923,-89,1730,68736,5646,64973.0,86332.0,151305,1432221,907022.0
1,2020-11-23T17:00:00,2,Valle d'Aosta,146,13,159,1690,1849,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0
2,2020-11-23T17:00:00,3,Lombardia,8331,945,9276,140642,149918,-14488,5289,206317,20664,280138.0,96761.0,376899,3831713,2262985.0
3,2020-11-23T17:00:00,5,Veneto,2280,290,2570,71157,73727,1271,2540,48237,3258,30561.0,94661.0,125222,2683388,1030974.0
4,2020-11-23T17:00:00,6,Friuli Venezia Giulia,565,55,620,12585,13205,154,377,11157,673,21232.0,3803.0,25035,667212,265792.0
5,2020-11-23T17:00:00,7,Liguria,1248,122,1370,13951,15321,-557,285,30830,2261,33202.0,15210.0,48412,567597,280174.0
6,2020-11-23T17:00:00,8,Emilia-Romagna,2738,248,2986,66394,69380,2106,2347,33309,5384,68021.0,40052.0,108073,2002154,1033714.0
7,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0
8,2020-11-23T17:00:00,10,Umbria,371,78,449,10034,10483,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0
9,2020-11-23T17:00:00,11,Marche,550,90,640,14822,15462,95,161,10105,1188,20547.0,6208.0,26755,392339,228318.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0
1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0
2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0
3,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0
4,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0
5,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0
6,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0
7,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0
8,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0
9,2020-11-22T17:00:00,11,Marche,542,88,630,14737,15367,245,529,10047,1180,20395.0,6199.0,26594,391100,227685.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-23T17:00:00,1,Piemonte,5112,399,5511,71412,76923,-89,1730,68736,5646,64973.0,86332.0,151305,1432221,907022.0
1,2020-11-23T17:00:00,2,Valle d'Aosta,146,13,159,1690,1849,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0
2,2020-11-23T17:00:00,3,Lombardia,8331,945,9276,140642,149918,-14488,5289,206317,20664,280138.0,96761.0,376899,3831713,2262985.0
3,2020-11-23T17:00:00,4,Trentino-Alto Adige,906,81,987,13146,14133,-33,353,20493,1059,28820.0,6865.0,35685,649891,273826.0
4,2020-11-23T17:00:00,5,Veneto,2280,290,2570,71157,73727,1271,2540,48237,3258,30561.0,94661.0,125222,2683388,1030974.0
5,2020-11-23T17:00:00,6,Friuli Venezia Giulia,565,55,620,12585,13205,154,377,11157,673,21232.0,3803.0,25035,667212,265792.0
6,2020-11-23T17:00:00,7,Liguria,1248,122,1370,13951,15321,-557,285,30830,2261,33202.0,15210.0,48412,567597,280174.0
7,2020-11-23T17:00:00,8,Emilia-Romagna,2738,248,2986,66394,69380,2106,2347,33309,5384,68021.0,40052.0,108073,2002154,1033714.0
8,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0
9,2020-11-23T17:00:00,10,Umbria,371,78,449,10034,10483,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-22T17:00:00,1,Piemonte,5132,398,5530,71482,77012,920,2641,66998,5565,64386.0,85189.0,149575,1418692,898135.0
1,1,2020-11-22T17:00:00,2,Valle d'Aosta,144,14,158,1761,1919,-82,99,3818,285,5364.0,658.0,6022,54841,31620.0
2,2,2020-11-22T17:00:00,3,Lombardia,8391,949,9340,155066,164406,1721,5094,186680,20524,276322.0,95288.0,371610,3798851,2248813.0
3,19,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,20119,1047,28514.0,6818.0,35332,646944,272866.0
4,3,2020-11-22T17:00:00,5,Veneto,2211,281,2492,69964,72456,1709,2956,47005,3221,30237.0,92445.0,122682,2672240,1027321.0
5,4,2020-11-22T17:00:00,6,Friuli Venezia Giulia,545,56,601,12450,13051,294,675,10961,646,20886.0,3772.0,24658,664296,264942.0
6,5,2020-11-22T17:00:00,7,Liguria,1253,122,1375,14503,15878,5,611,30008,2241,33196.0,14931.0,48127,564766,279225.0
7,6,2020-11-22T17:00:00,8,Emilia-Romagna,2666,249,2915,64359,67274,2194,2665,33105,5350,66834.0,38895.0,105729,1990596,1027297.0
8,7,2020-11-22T17:00:00,9,Toscana,1785,298,2083,50641,52724,-416,1929,39765,2216,73871.0,20834.0,94705,1457791,912549.0
9,8,2020-11-22T17:00:00,10,Umbria,368,75,443,11129,11572,149,400,9864,333,6133.0,15636.0,21769,390378,215754.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-23T17:00:00,1,Piemonte,5112,399,5511,71412,76923,-89,1730,68736,5646,64973.0,86332.0,151305,1432221,907022.0,4341375
1,2020-11-23T17:00:00,2,Valle d'Aosta,146,13,159,1690,1849,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0,125501
2,2020-11-23T17:00:00,3,Lombardia,8331,945,9276,140642,149918,-14488,5289,206317,20664,280138.0,96761.0,376899,3831713,2262985.0,10103969
3,2020-11-23T17:00:00,4,Trentino-Alto Adige,906,81,987,13146,14133,-33,353,20493,1059,28820.0,6865.0,35685,649891,273826.0,1074819
4,2020-11-23T17:00:00,5,Veneto,2280,290,2570,71157,73727,1271,2540,48237,3258,30561.0,94661.0,125222,2683388,1030974.0,4907704
5,2020-11-23T17:00:00,6,Friuli Venezia Giulia,565,55,620,12585,13205,154,377,11157,673,21232.0,3803.0,25035,667212,265792.0,1211357
6,2020-11-23T17:00:00,7,Liguria,1248,122,1370,13951,15321,-557,285,30830,2261,33202.0,15210.0,48412,567597,280174.0,1543127
7,2020-11-23T17:00:00,8,Emilia-Romagna,2738,248,2986,66394,69380,2106,2347,33309,5384,68021.0,40052.0,108073,2002154,1033714.0,4467118
8,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0,3722729
9,2020-11-23T17:00:00,10,Umbria,371,78,449,10034,10483,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-23T17:00:00,1,Piemonte,5112,399,5511,71412,76923,-89,1730,68736,5646,64973.0,86332.0,151305,1432221,907022.0,4341375,575
1,2020-11-23T17:00:00,2,Valle d'Aosta,146,13,159,1690,1849,-70,77,3960,290,5434.0,665.0,6099,55507,31934.0,125501,20
2,2020-11-23T17:00:00,3,Lombardia,8331,945,9276,140642,149918,-14488,5289,206317,20664,280138.0,96761.0,376899,3831713,2262985.0,10103969,1036
3,2020-11-23T17:00:00,4,Trentino-Alto Adige,906,81,987,13146,14133,-33,353,20493,1059,28820.0,6865.0,35685,649891,273826.0,1074819,106
4,2020-11-23T17:00:00,5,Veneto,2280,290,2570,71157,73727,1271,2540,48237,3258,30561.0,94661.0,125222,2683388,1030974.0,4907704,1016
5,2020-11-23T17:00:00,6,Friuli Venezia Giulia,565,55,620,12585,13205,154,377,11157,673,21232.0,3803.0,25035,667212,265792.0,1211357,180
6,2020-11-23T17:00:00,7,Liguria,1248,122,1370,13951,15321,-557,285,30830,2261,33202.0,15210.0,48412,567597,280174.0,1543127,209
7,2020-11-23T17:00:00,8,Emilia-Romagna,2738,248,2986,66394,69380,2106,2347,33309,5384,68021.0,40052.0,108073,2002154,1033714.0,4467118,634
8,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,41345,2264,74667.0,21361.0,96028,1468342,917080.0,3722729,523
9,2020-11-23T17:00:00,10,Umbria,371,78,449,10034,10483,-1089,105,11054,337,6158.0,15716.0,21874,390826,215945.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-23T17:00:00,3,Lombardia,8331,945,9276,140642,149918,-14488,5289,...,91.22,-14424,195,32862,14172.0,140,19637,16.09,0.05235,3.73
1,2020-11-23T17:00:00,5,Veneto,2280,290,2570,71157,73727,1271,2540,...,28.54,1193,-416,11148,3653.0,37,1232,22.78,0.05176,2.55
2,2020-11-23T17:00:00,8,Emilia-Romagna,2738,248,2986,66394,69380,2106,2347,...,39.12,2035,-318,11558,6417.0,34,204,20.31,0.05254,2.42
3,2020-11-23T17:00:00,12,Lazio,3351,335,3686,80203,83889,1836,2341,...,39.55,1787,-192,20324,16324.0,48,457,11.52,0.03991,1.79
4,2020-11-23T17:00:00,15,Campania,2331,201,2532,100977,103509,28,2158,...,39.80,-85,-1059,15739,6479.0,39,2091,13.71,0.03730,2.39
5,2020-11-23T17:00:00,1,Piemonte,5112,399,5511,71412,76923,-89,1730,...,69.39,-70,-911,13529,8887.0,81,1738,12.79,0.03985,3.49
6,2020-11-23T17:00:00,9,Toscana,1832,296,2128,50291,52419,-305,1323,...,56.60,-350,-606,10551,4531.0,48,1580,12.54,0.03554,2.58
7,2020-11-23T17:00:00,19,Sicilia,1604,243,1847,36066,37913,751,1249,...,41.33,742,-9,7712,4984.0,41,457,16.20,0.02514,1.09
8,2020-11-23T17:00:00,16,Puglia,1499,191,1690,30289,31979,627,980,...,51.76,574,-347,3869,1550.0,40,313,25.33,0.02445,1.11
9,2020-11-23T17:00:00,13,Abruzzo,672,71,743,15606,16349,218,640,...,47.97,211,80,4491,1958.0,18,404,14.25,0.04901,1.86


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,945,91,1036,91.22
1,Trentino-Alto Adige,81,25,106,76.42
2,Piemonte,399,176,575,69.39
3,Umbria,78,39,117,66.67
4,Valle d'Aosta,13,7,20,65.00
5,Marche,90,53,143,62.94
6,Liguria,122,87,209,58.37
7,Toscana,296,227,523,56.60
8,Puglia,191,178,369,51.76
9,Abruzzo,71,77,148,47.97


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,2020-11-21T17:00:00,22,P.A. Trento,413,40,453,2249,2702,-302,212,10694,577,7252.0,6721.0,13973,348903,127347.0
544,2020-11-22T17:00:00,21,P.A. Bolzano,453,38,491,11133,11624,-51,535,9107,469,21200.0,0.0,21200,294506,144616.0
545,2020-11-22T17:00:00,22,P.A. Trento,420,41,461,2081,2542,-160,159,11012,578,7314.0,6818.0,14132,352438,128250.0
546,2020-11-23T17:00:00,21,P.A. Bolzano,473,38,511,11104,11615,-9,260,9371,474,21460.0,0.0,21460,295974,145053.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,...,20.0,6135.0,1790.0,431.0,8.18,6.72,15.68,53.74,1.30,6.991
270,2020-11-20T17:00:00,4,Trentino-Alto Adige,885,76,961,13554,14515,325,970,...,15.0,6330.0,1663.0,318.0,7.91,6.62,15.32,58.33,-2.56,6.836
271,2020-11-21T17:00:00,4,Trentino-Alto Adige,863,79,942,13435,14377,-138,760,...,30.0,7784.0,2663.0,-119.0,8.39,6.55,9.76,28.54,3.95,5.236
272,2020-11-22T17:00:00,4,Trentino-Alto Adige,873,79,952,13214,14166,-211,694,...,8.0,6424.0,1844.0,-221.0,8.30,6.72,10.80,37.64,0.00,4.827


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
